In [1]:
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('flights.jan.csv')

In [3]:
print('Dimensões do dataframe: ', df.shape)

Dimensões do dataframe:  (469968, 32)


In [15]:
# informações de valores nulos e tipos das colunas

tab_info=pd.DataFrame(df.dtypes).T.rename(index={0:'tipo de dado'})

tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'valores nulos (contagem)'}))

tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100).T.rename(index={0:'valores nulos (%)'}))
tab_info

/var/folders/m6/535gs6dj4pl874nwyz4wq4340000gn/T/ipykernel_7065/3741533118.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'valores nulos (contagem)'}))
/var/folders/m6/535gs6dj4pl874nwyz4wq4340000gn/T/ipykernel_7065/3741533118.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100).T.rename(index={0:'valores nulos (%)'}))


,Unnamed: 0,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
tipo de dado,int64,int64,int64,int64,int64,object,int64,object,object,object,...,float64,float64,int64,int64,object,float64,float64,float64,float64,float64
valores nulos (contagem),0,0,0,0,0,0,0,2782,0,0,...,12271,12955,0,0,457986,374017,374017,374017,374017,374017
valores nulos (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.591955,0.0,0.0,...,2.611029,2.756571,0.0,0.0,97.450465,79.583504,79.583504,79.583504,79.583504,79.583504


In [16]:
df['DATE'] = pd.to_datetime(df[['YEAR','MONTH', 'DAY']])

In [19]:
# Função que converte a string 'HHMM' em datetime.time

def formatar_horario(dado_orig):
  if pd.isnull(dado_orig):
    return np.nan
  else:
    if dado_orig == 2400: dado_orig = 0
    dado_orig = "{0:04d}".format(int(dado_orig))
    horario = datetime.time(int(dado_orig[0:2]), int(dado_orig[2:4]))
    return horario


# Função que combina a data e o horario para criar um datetime.datetime

def combinar_data_horario(x):
  if pd.isnull(x[0]) or pd.isnull(x[1]):
    return np.nan
  else:
    return datetime.datetime.combine(x[0],x[1])


# Função que combina duas colunas do dataframe para criar um formato datetime

def criar_horario_voo(df, col):    
  lista = []
  for index, cols in df[['DATE', col]].iterrows():    
    if pd.isnull(cols[1]):
      lista.append(np.nan)
    elif float(cols[1]) == 2400:
      cols[0] += datetime.timedelta(days=1)
      cols[1] = datetime.time(0,0)
      lista.append(combinar_data_horario(cols))
    else:
      cols[1] = formatar_horario(cols[1])
      lista.append(combinar_data_horario(cols))
  return pd.Series(lista)

In [20]:
df['SCHEDULED_DEPARTURE'] = criar_horario_voo(df, 'SCHEDULED_DEPARTURE')
df['DEPARTURE_TIME'] = df['DEPARTURE_TIME'].apply(formatar_horario)
df['SCHEDULED_ARRIVAL'] = df['SCHEDULED_ARRIVAL'].apply(formatar_horario)
df['ARRIVAL_TIME'] = df['ARRIVAL_TIME'].apply(formatar_horario)

In [21]:
df.loc[:5, ['SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL', 'DEPARTURE_TIME', 'ARRIVAL_TIME', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']]

,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL,DEPARTURE_TIME,ARRIVAL_TIME,DEPARTURE_DELAY,ARRIVAL_DELAY
0,2015-01-01 00:05:00,04:30:00,23:54:00,04:08:00,-11.0,-22.0
1,2015-01-01 00:10:00,07:50:00,00:02:00,07:41:00,-8.0,-9.0
2,2015-01-01 00:20:00,08:06:00,00:18:00,08:11:00,-2.0,5.0
3,2015-01-01 00:20:00,08:05:00,00:15:00,07:56:00,-5.0,-9.0
4,2015-01-01 00:25:00,03:20:00,00:24:00,02:59:00,-1.0,-21.0
5,2015-01-01 00:25:00,06:02:00,00:20:00,06:10:00,-5.0,8.0
